In [4]:
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
from django.core.exceptions import ObjectDoesNotExist, FieldError
file = "archiv/data/Database_Vorarlberg.xlsx"
df = pd.read_excel(file, "RESEARCH")[2:].fillna('False')

In [6]:
failed_sites = []
for i, row in df.iterrows():
    temp_item = None
    try:
        temp_site = Site.objects.get(identifier=row['*SiteID'])
    except ObjectDoesNotExist:
        failed_sites.append(row['*SiteID'])
        temp_site = None
    if temp_site:
        if row['*Research Activity ID'] != 'False':
            temp_item, _ = ResearchEvent.objects.get_or_create(
                legacy_research_id=row['*Research Activity ID']
            )
            temp_item.site_id.add(temp_site)
            

        if row['Start Date'] != 'False':
            
            try:
                start_date = datetime.strptime(str(row['Start Date']), '%d%m%Y')
            except ValueError:
                start_date = None
            if start_date:
                temp_item.start_date = start_date
            else:
                try:
                    temp_item.start_date = pd.to_datetime(row['Start Date'])
                except ValueError:
                    pass
        
        if row['End Date'] != 'False':
            try:
                end_date = datetime.strptime(str(row['End Date']), '%d%m%Y')
            except ValueError:
                end_date = None
            if end_date:
                temp_item.end_date = end_date
            else:
                try:
                    temp_item.end_date = pd.to_datetime(row['End Date'])
                except ValueError:
                    pass
        
        
        if row['Responsible Researcher'] != 'False':
            x = None
            temp_person = None
            for x in row['Responsible Researcher'].split(','):
                x = x.strip()
                temp_person, _ = Person.objects.get_or_create(written_name=x)
                if len(x.split(' ')) == 2:
                    temp_person.forename = x.split(' ')[0]
                    temp_person.name = x.split(' ')[1]
                elif len(x.split(' ')) == 1:
                    temp_person.name = x
                temp_person.save()
                temp_item.responsible_researcher.add(temp_person)
        
        if row['Responsible Institution'] != 'False':
            x = None
            temp_inst = None
            for x in row['Responsible Institution'].split(','):
                x = x.strip()
                temp_inst, _ = Institution.objects.get_or_create(legacy_id=x)
                temp_inst.written_name = x
                temp_inst.save()
                temp_item.responsible_institution.add(temp_inst)
        
        if row['Activity-ID'] != 'False':
            temp_item.identifier = "{}".format(row['Activity-ID'])
        
        if row['Research Question'] != 'False':
            rq, _ = ResearchQuestion.objects.get_or_create(question=row['Research Question'])
            temp_item.research_question = rq
        
        if row['Comment'] != 'False':
            temp_item.comment = row['Comment']
            
        if row['Research type'] != 'False':
            try:
                x = SkosConcept.objects.get(pref_label=row['Research type'].lower())
            except ObjectDoesNotExist:
                x = None
            if x:
                temp_item.research_type = x
        
        if row['Research Method'] != 'False':
            x = None
            for x in row['Research Method'].split(','):
                x = x.strip()
                try:
                    x = SkosConcept.objects.get(pref_label=x.lower())
                except ObjectDoesNotExist:
                    x = None
                if x:
                    temp_item.research_method.add(x)
        temp_item.save()

In [ ]:
# ResearchEvent.objects.all().delete()